In [ ]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import scipy.signal as sig
import simpleaudio

# Configuration constants
PLAY_SOUND = False

handel = loadmat("handel.mat")
fs = handel["Fs"][0][0]
handel = handel["y"]
print(type(handel[0][0]))
print(handel.shape)
# Convert to a flat vector
print(handel.T.shape)
handel = handel.T[0]
plt.figure()
plt.plot(handel)
plt.xlabel("Sample number")
plt.ylabel("Amplitude")
plt.show()


In [ ]:
# simpleaudio is simple, but this means limited support for what it deems "weird" data
# The data size returned by loadmat and the sample rate are both "weird"
# Rather than determine a good method for interpolation/resampling,
# We're just going to lie and say 8192Hz ~= 8000Hz, a supported sample rate
# Just consider it Vaporwave or something (Baroquewave?)
if PLAY_SOUND:
    simpleaudio.play_buffer(handel.astype(np.float32), 1, 4, 8000)

In [ ]:
def get_windows(data, width, overlap, copy=True):
    """
    See here: https://stackoverflow.com/a/45730836
    """
    sh = (data.size - width + 1, width)
    st = data.strides * 2
    view = np.lib.stride_tricks.as_strided(data.T, strides=st, shape=sh)[0::overlap]
    print("View shape: {}".format(view.shape))
    if copy:
        return view.copy()
    else:
        return view
    
def apply_window_function(windowed_data, windowing_function):
    # Not pythonic to LBYL but this could be slow so thbbbtttt
    if windowing_function.ndim != 1:
        raise ValueError("Windowing function must be 1 dimensional!")
    if windowed_data.ndim != 2:
        raise ValueError("Windowed data must be 2D ndarray!")
    if windowed_data.shape[1] != windowing_function.shape[0]:
        print("windowed_data.shape: {}, windowing_function.shape: {}".format(windowed_data.shape, windowing_function.shape))
        raise ValueError("Windowing function length != data window length!")

    return windowed_data*windowing_function


def spectrogram(data, fs=1.0, window=None, nperseg=None, noverlap=None):
    """
    Return the calculated spectrogram in a fashion similar to SciPy's API.
    Assumes input data is real, therefore the FFT is symettric, thus returning only
    the frequencies >=0. 
    """
    if data.ndim != 1:
        raise ValueError("We only support 1D data.")
    if nperseg is None:
        nperseg = 256
    if nperseg > data.size:
        warnings.warn("nperseg larger than data, reducing...")
        nperseg = data.size
    if noverlap is None:
        noverlap = np.int(np.floor(nperseg / 8))
        
    if window is None:
        # Shannon/Boxcar/Square/Dirichlet Window (or 'no' window)
        window = np.ones(nperseg)
        
    if window.ndim != 1:
        raise ValueError("Window must be 1D!")
    if window.size != nperseg:
        print("nperseg: {}, window.size: {}".format(nperseg, window.size))
        raise ValueError("Windowing function must be the size of the window!")
    
    chunked_data = get_windows(data, nperseg, noverlap)
    windowed_data = apply_window_function(chunked_data, window)
    
    t = np.arange(nperseg/2, data.shape[-1] - nperseg/2+1, nperseg-noverlap)/float(fs)
    freqs = (np.arange(1, nperseg + 1, dtype=int) // 2) / float(nperseg*(1.0/fs))
    # the above produces 2 of every frequency but the 0 and max frequency, so take the uniques
    # Slicing proved annoying to get consistent. probably not the most performant but WHATEVAH
    freqs = np.unique(freqs)
    Sxx = np.apply_along_axis(np.fft.rfft, 1, windowed_data).T
    
    return freqs, t, Sxx
    

def plot_spectrogram(data, fs, window=None, nperseg=None, noverlap=None, native=False):
    if native:
        # Just for validation purposes
        f, t, Sxx = sig.spectrogram(data, fs=fs, window=window, nperseg=nperseg, noverlap=noverlap)
    else:
        f, t, Sxx = spectrogram(data, fs=fs, window=window, nperseg=nperseg, noverlap=noverlap)
    plt.figure()
    plt.pcolormesh(t, f, Sxx)
    plt.ylabel("Frequency (Hz)")
    plt.xlabel("Time (Seconds)")
    plt.show()

In [ ]:
# generate a shannon window for consistency in validation
nperseg = 256
noverlap = nperseg // 8
shannon = np.ones(nperseg)
m_f, m_t, m_Sxx = spectrogram(handel, 8192, window=shannon, nperseg=nperseg, noverlap=noverlap)
f, t, Sxx = sig.spectrogram(handel, 8192, window=shannon, nperseg=nperseg, noverlap=noverlap)

In [ ]:
Sxx.shape

In [ ]:
m_Sxx.shape